# tushare for python2

In [18]:
import tushare as ts

- 历史行情数据
- 复权历史数据
- 实时行情数据
- 历史分笔数据
- 实时报价数据
- 当日历史分笔
- 大盘指数列表
- 大单交易数据

In [1]:
# -*- coding:utf-8 -*- 
"""
交易数据接口 
Created on 2014/07/31
@author: Jimmy Liu
@group : waditu
@contact: jimmysoa@sina.cn
"""
from __future__ import division

import time
import json
import lxml.html
from lxml import etree
import pandas as pd
import numpy as np
import datetime
import re
from pandas.compat import StringIO
import os
try:
    from urllib.request import urlopen, Request
except ImportError:
    from urllib2 import urlopen, Request

In [6]:
from tushare.stock import cons as ct
from tushare.util.conns import get_apis, close_apis
from tushare.stock.fundamental import get_stock_basics
from tushare.util import dateu as du
from tushare.util.formula import MA

In [7]:
def get_hist_data(code=None, start=None, end=None,
                  ktype='D', retry_count=3,
                  pause=0.001):
    """
        获取个股历史交易记录
    Parameters
    ------
      code:string
                  股票代码 e.g. 600848
      start:string
                  开始日期 format：YYYY-MM-DD 为空时取到API所提供的最早日期数据
      end:string
                  结束日期 format：YYYY-MM-DD 为空时取到最近一个交易日数据
      ktype：string
                  数据类型，D=日k线 W=周 M=月 5=5分钟 15=15分钟 30=30分钟 60=60分钟，默认为D
      retry_count : int, 默认 3
                 如遇网络等问题重复执行的次数 
      pause : int, 默认 0
                重复请求数据过程中暂停的秒数，防止请求间隔时间太短出现的问题
    return
    -------
      DataFrame
          属性:日期 ，开盘价， 最高价， 收盘价， 最低价， 成交量， 价格变动 ，涨跌幅，5日均价，10日均价，20日均价，5日均量，10日均量，20日均量，换手率
    """
    symbol = ct._code_to_symbol(code)
    url = ''
    if ktype.upper() in ct.K_LABELS:
        url = ct.DAY_PRICE_URL%(ct.P_TYPE['http'], ct.DOMAINS['ifeng'],
                                ct.K_TYPE[ktype.upper()], symbol)
    elif ktype in ct.K_MIN_LABELS:
        url = ct.DAY_PRICE_MIN_URL%(ct.P_TYPE['http'], ct.DOMAINS['ifeng'],
                                    symbol, ktype)
    else:
        raise TypeError('ktype input error.')
    
    for _ in range(retry_count):
        time.sleep(pause)
        try:
            request = Request(url)
            lines = urlopen(request, timeout = 10).read()
            if len(lines) < 15: #no data
                return None
        except Exception as e:
            print(e)
        else:
            js = json.loads(lines.decode('utf-8') if ct.PY3 else lines)
            cols = []
            if (code in ct.INDEX_LABELS) & (ktype.upper() in ct.K_LABELS):
                cols = ct.INX_DAY_PRICE_COLUMNS
            else:
                cols = ct.DAY_PRICE_COLUMNS
            if len(js['record'][0]) == 14:
                cols = ct.INX_DAY_PRICE_COLUMNS
            df = pd.DataFrame(js['record'], columns=cols)
            if ktype.upper() in ['D', 'W', 'M']:
                df = df.applymap(lambda x: x.replace(u',', u''))
                df[df==''] = 0
            for col in cols[1:]:
                df[col] = df[col].astype(float)
            if start is not None:
                df = df[df.date >= start]
            if end is not None:
                df = df[df.date <= end]
            if (code in ct.INDEX_LABELS) & (ktype in ct.K_MIN_LABELS):
                df = df.drop('turnover', axis=1)
            df = df.set_index('date')
            df = df.sort_index(ascending = False)
            return df
    raise IOError(ct.NETWORK_URL_ERROR_MSG)

In [10]:
def get_k_data(code=None, start='', end='',
                  ktype='D', autype='qfq', 
                  index=False,
                  retry_count=3,
                  pause=0.001):
    """
    获取k线数据
    ---------
    Parameters:
      code:string
                  股票代码 e.g. 600848
      start:string
                  开始日期 format：YYYY-MM-DD 为空时取上市首日
      end:string
                  结束日期 format：YYYY-MM-DD 为空时取最近一个交易日
      autype:string
                  复权类型，qfq-前复权 hfq-后复权 None-不复权，默认为qfq
      ktype：string
                  数据类型，D=日k线 W=周 M=月 5=5分钟 15=15分钟 30=30分钟 60=60分钟，默认为D
      retry_count : int, 默认 3
                 如遇网络等问题重复执行的次数 
      pause : int, 默认 0
                重复请求数据过程中暂停的秒数，防止请求间隔时间太短出现的问题
    return
    -------
      DataFrame
          date 交易日期 (index)
          open 开盘价
          high  最高价
          close 收盘价
          low 最低价
          volume 成交量
          amount 成交额
          turnoverratio 换手率
          code 股票代码
    """
    symbol = ct.INDEX_SYMBOL[code] if index else ct._code_to_symbol(code)
    url = ''
    dataflag = ''
    autype = '' if autype is None else autype
    if (start is not None) & (start != ''):
        end = du.today() if end is None or end == '' else end
    if ktype.upper() in ct.K_LABELS:
        fq = autype if autype is not None else ''
        if code[:1] in ('1', '5') or index:
            fq = ''
        kline = '' if autype is None else 'fq'
        if (start is None or start == '') & (end is None or end == ''):
            urls = [ct.KLINE_TT_URL%(ct.P_TYPE['http'], ct.DOMAINS['tt'],
                                    kline, fq, symbol, 
                                    ct.TT_K_TYPE[ktype.upper()], start, end,
                                    fq, _random(17))]
        else:
            years = du.tt_dates(start, end)
            urls = []
            for year in years:
                startdate = str(year) + '-01-01'
                enddate = str(year+1) + '-12-31'
                url = ct.KLINE_TT_URL%(ct.P_TYPE['http'], ct.DOMAINS['tt'],
                                    kline, fq+str(year), symbol, 
                                    ct.TT_K_TYPE[ktype.upper()], startdate, enddate,
                                    fq, _random(17))
                urls.append(url)
        dataflag = '%s%s'%(fq, ct.TT_K_TYPE[ktype.upper()])
    elif ktype in ct.K_MIN_LABELS:
        urls = [ct.KLINE_TT_MIN_URL%(ct.P_TYPE['http'], ct.DOMAINS['tt'],
                                    symbol, ktype, ktype,
                                    _random(16))]
        dataflag = 'm%s'%ktype
    else:
        raise TypeError('ktype input error.')
    data = pd.DataFrame()
    for url in urls:
        data = data.append(_get_k_data(url, dataflag, 
                                       symbol, code,
                                       index, ktype,
                                       retry_count, pause), 
                           ignore_index=True)
    if ktype not in ct.K_MIN_LABELS:
        if ((start is not None) & (start != '')) & ((end is not None) & (end != '')):
            if data.empty==False:       
                data = data[(data.date >= start) & (data.date <= end)]
    return data
    raise IOError(ct.NETWORK_URL_ERROR_MSG)

In [12]:
def _random(n=13):
    from random import randint
    start = 10**(n-1)
    end = (10**n)-1
    return str(randint(start, end))

In [14]:
def _get_k_data(url, dataflag='',
                symbol='',
                code = '',
                index = False,
                ktype = '',
                retry_count=3,
                pause=0.001):
    for _ in range(retry_count):
            time.sleep(pause)
            try:
                request = Request(url)
                lines = urlopen(request, timeout = 10).read()
                if len(lines) < 100: #no data
                    return None
            except Exception as e:
                print(e)
            else:
                lines = lines.decode('utf-8') if ct.PY3 else lines
                lines = lines.split('=')[1]
                reg = re.compile(r',{"nd.*?}') 
                lines = re.subn(reg, '', lines) 
                js = json.loads(lines[0])
                dataflag = dataflag if dataflag in list(js['data'][symbol].keys()) else ct.TT_K_TYPE[ktype.upper()]
                if len(js['data'][symbol][dataflag]) == 0:
                    return None
                if len(js['data'][symbol][dataflag][0]) == 6:
                    df = pd.DataFrame(js['data'][symbol][dataflag], 
                                  columns = ct.KLINE_TT_COLS_MINS)
                else:
                    df = pd.DataFrame(js['data'][symbol][dataflag], 
                                  columns = ct.KLINE_TT_COLS)
                df['code'] = symbol if index else code
                if ktype in ct.K_MIN_LABELS:
                    df['date'] = df['date'].map(lambda x: '%s-%s-%s %s:%s'%(x[0:4], x[4:6], 
                                                                            x[6:8], x[8:10], 
                                                                            x[10:12]))
                for col in df.columns[1:6]:
                    df[col] = df[col].astype(float)
                return df

In [8]:
# 历史行情
get_hist_data('300676', start='2018-01-01', end='2018-07-31') #一次性获取全部日k线数据 
# failed

In [17]:
# get_k_data('300676')

In [18]:
# 复权数据
df = ts.get_stock_basics()
date = df.ix['600848']['timeToMarket'] #上市日期YYYYMMDD

In [21]:
df.ix['600848']

name                       上海临港
industry                   园区开发
area                         上海
pe                         90.9
outstanding                   4
totals                     11.2
totalAssets         1.39677e+06
liquidAssets        1.00801e+06
fixedAssets             12230.9
reserved                 423993
reservedPerShare           3.79
esp                        0.06
bvps                       5.87
pb                         3.69
timeToMarket           19940324
undp                     110493
perundp                    0.99
rev                      -19.95
profit                    19.21
gpr                       54.32
npr                       23.88
holders                   61764
Name: 600848, dtype: object

## 获取某家公司的财务信息

In [4]:
all = ts.get_stock_basics()
all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3538 entries, 600225 to 000950
Data columns (total 22 columns):
name                3538 non-null object
industry            3538 non-null object
area                3538 non-null object
pe                  3538 non-null float64
outstanding         3538 non-null float64
totals              3538 non-null float64
totalAssets         3538 non-null float64
liquidAssets        3538 non-null float64
fixedAssets         3538 non-null float64
reserved            3538 non-null float64
reservedPerShare    3538 non-null float64
esp                 3538 non-null float64
bvps                3538 non-null float64
pb                  3538 non-null float64
timeToMarket        3538 non-null int64
undp                3538 non-null float64
perundp             3538 non-null float64
rev                 3538 non-null float64
profit              3538 non-null float64
gpr                 3538 non-null float64
npr                 3538 non-null float64
holders       

In [11]:
all.loc["300676"]

name                    华大基因
industry                医疗保健
area                      深圳
pe                     72.59
outstanding             2.47
totals                     4
totalAssets           504989
liquidAssets          369130
fixedAssets          77748.9
reserved              296826
reservedPerShare        7.42
esp                    0.252
bvps                   10.54
pb                      6.93
timeToMarket        20170714
undp                 80426.1
perundp                 2.01
rev                    33.13
profit                  6.71
gpr                    53.68
npr                    19.51
holders                37289
Name: 300676, dtype: object

# 获取行业分类信息

In [19]:
ts.get_industry_classified()

,code,name,c_name
0,600051,宁波联合,综合行业
1,600209,罗顿发展,综合行业
2,600212,江泉实业,综合行业
3,600256,广汇能源,综合行业
4,600576,祥源文化,综合行业
5,600603,广汇物流,综合行业
6,600614,鹏起科技,综合行业
7,600620,天宸股份,综合行业
8,600622,光大嘉宝,综合行业
9,600624,复旦复华,综合行业


# 存储为csv格式，在R中做下游分析

## 我需要哪些数据？
- 表1：每个公司所属的行业信息，以及其他分类标准；（静态）
- 表2：每个公司的财务信息；（动态，季度）
- 表3：各种指数的信息（国内外各种指数，期货）；（动态，每日）
- 表4：每个公司的每日的收盘价和成交量；（动态，每日）
- 表5：

# 0.过滤掉上市不足2年的股票，过滤掉创业板，过滤掉小盘股

## 1.选出0.25、0.5、1、2年内每个股所处的价位比，比如如果某股在0.5年内处于（max-min）的0%，那它就是半年内一直跌
我比较喜欢那种已经大跌过，但现在开始启动行情的个股。

# 2.每个股票之间的相关性分析，以及其与大盘、外汇、外盘的相关性分析。找出一些股票模块。 

# 3.根据财务指标对每个行业的个股进行排序，不碰行业的吊车尾。不碰有太多负债、恶性资产且没有盈利能力的公司。不碰即将解禁、做空势力大的股票（现在的华大）。

# 4.开发一款工具来检测启动/反势行情

# 5.最终肯定会筛选出一些个股，需要建立一套完善的个股评价标准，确定最终是否应该买入。

# 6.交易策略的历史验证

# 7.永远不要做纯粹主观的交易

# 8.个股的估值

[美股估值的三种常用方法 一看就懂](http://stock.qq.com/a/20160317/058536.htm)